## Upload covid data to Socrata

References:

* [socrata-py package](https://github.com/socrata/socrata-py)
* [sodapy package](https://pypi.org/project/sodapy/)
* [Socrata Data Management experience](https://support.socrata.com/hc/en-us/articles/115016067067-Using-the-Socrata-Data-Management-Experience)
* [some examples](https://github.com/xmunoz/sodapy)

In [1]:
import pandas as pd

from processing_utils import default_parameters
from processing_utils import neighborhood_utils
from processing_utils import utils

import credentials

S3_FILE_PATH = default_parameters.S3_FILE_PATH

SOCRATA_USER = credentials.SOCRATA_USERNAME
SOCRATA_PASSWORD = credentials.SOCRATA_PASSWORD

In [4]:
DATAFRAME_DICT = {
    # key: socrata_dataset_id
    # value: csv file
    "jsff-uc6b": "us-county-time-series.csv",
    "fvye-93wd": "la-county-neighborhood-time-series.csv",
    "rpp7-mevy": "vaccinations-by-county.csv",
    "iv7a-6rrq": "vaccinations-by-demographics-county.csv"
}

for socrata_id, filename in DATAFRAME_DICT.items():
    # So far, all the datasets need some extra processing to make sure Socrata table schema is correct
    if (("us-county" in filename) or 
        ("neighborhood" in filename) or 
        ("vaccinations" in filename)):
        df = extra_processing(filename)
    else:
        df = pd.read_csv(f"{S3_FILE_PATH}{filename}")
    
    # Write the full table out as CSV 
    df.to_csv(f"{filename}", index=False)
    
    #overwrite_socrata_table(filename, socrata_dataset_id = socrata_id)
    socrata_utils.upsert_socrata_rows(SOCRATA_USER, SOCRATA_PASSWORD, 
                                       filename, socrata_dataset_id = socrata_id)
    
    
#s3_to_socrata("la-county-neighborhood-time-series.csv", 
#              socrata_dataset_id = "wrj6-vjm7")

us-county-time-series.csv updated
la-county-neighborhood-time-series.csv updated
vaccinations-by-county.csv updated
vaccinations-by-demographics-county.csv updated


In [33]:
'''
la = pd.read_csv("../notebooks/test_data/us-county-time-series.csv")
la = la[la.county=="Los Angeles"]
la[la.date < "2021-08-23"].to_csv("../notebooks/test_data/la.csv", index=False)
la.to_csv("../notebooks/test_data/la.csv", index=False)

filename = "../notebooks/test_data/la.csv"
socrata_dataset_id = "3cst-kzzr"
existing_table = client.get(socrata_dataset_id)
existing_table = pd.DataFrame(existing_table)
max_date = pd.to_datetime(existing_table.date.max())

t2 = pd.read_csv(f"{filename}")
t2 = t2.assign(
    date = pd.to_datetime(t2.date)
)

t2[t2.date > max_date].to_csv(filename, index=False)
data = open(f"{filename}")
    
client.timeout = (5 * 60)
client.upsert(socrata_dataset_id, data)

'''

{'Errors': 0, 'Rows Created': 2, 'Rows Deleted': 0, 'Rows Updated': 0}